# 🧬 AlphaFold vs ESMFold: Direct Comparison

## Side-by-Side Model Comparison

Analyzing the same proteins with both models:
- **High Confidence**: Human Ubiquitin (P0CG48)
- **Low Confidence**: Human Alpha-synuclein (P37840)

**Models Compared**:
- 🔵 AlphaFold (DeepMind) - MSA-based structure prediction
- 🟣 ESMFold (Meta AI) - Language model-based structure prediction

In [ ]:
%matplotlib inline

import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import tempfile
import os
import json

plt.style.use('seaborn-v0_8-darkgrid')
print("✅ Imports complete")

## Configuration

In [ ]:
# API Endpoints
ALPHAFOLD_API = "https://alphafold.ebi.ac.uk/api"
ALPHAFOLD_FILES = "https://alphafold.ebi.ac.uk/files"
ESMFOLD_API = "https://api.esmatlas.com/foldSequence/v1/pdb/"

# Proteins to compare
PROTEINS = {
    'ubiquitin': {'name': 'Human Ubiquitin', 'uniprot_id': 'P0CG48'},
    'synuclein': {'name': 'Human Alpha-Synuclein', 'uniprot_id': 'P37840'}
}

print("🔬 Comparing AlphaFold vs ESMFold on:")
for key, protein in PROTEINS.items():
    print(f"   • {protein['name']} ({protein['uniprot_id']})")

## Shared Functions

In [ ]:
def extract_plddt_from_pdb(pdb_file: str) -> Tuple[np.ndarray, List[int]]:
    plddt_scores, residue_numbers = [], []
    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith('ATOM') and ' CA ' in line:
                residue_numbers.append(int(line[22:26].strip()))
                plddt_scores.append(float(line[60:66].strip()))
    return np.array(plddt_scores), residue_numbers

def calculate_stats(plddt: np.ndarray) -> Dict:
    return {
        'mean': float(np.mean(plddt)),
        'std': float(np.std(plddt)),
        'very_high': float(np.sum(plddt > 90) / len(plddt) * 100),
        'high': float(np.sum((plddt > 70) & (plddt <= 90)) / len(plddt) * 100),
        'medium': float(np.sum((plddt > 50) & (plddt <= 70)) / len(plddt) * 100),
        'low': float(np.sum(plddt <= 50) / len(plddt) * 100),
    }

print("✅ Shared functions defined")

## AlphaFold Analysis

In [ ]:
def get_alphafold_structure(uniprot_id: str) -> str:
    # Get metadata
    meta_url = f"{ALPHAFOLD_API}/prediction/{uniprot_id}"
    response = requests.get(meta_url, timeout=30)
    response.raise_for_status()
    metadata = response.json()[0] if isinstance(response.json(), list) else response.json()
    version = metadata.get('latestVersion', 4)
    
    # Download PDB
    pdb_url = f"{ALPHAFOLD_FILES}/AF-{uniprot_id}-F1-model_v{version}.pdb"
    response = requests.get(pdb_url, timeout=60)
    response.raise_for_status()
    
    pdb_path = os.path.join(tempfile.gettempdir(), f"af_{uniprot_id}.pdb")
    with open(pdb_path, 'w') as f:
        f.write(response.text)
    return pdb_path

print("🔵 Running AlphaFold Analysis...\n")
alphafold_results = {}

for key, protein in PROTEINS.items():
    print(f"   AlphaFold: {protein['name']}...", end=' ')
    pdb_file = get_alphafold_structure(protein['uniprot_id'])
    plddt, res = extract_plddt_from_pdb(pdb_file)
    stats = calculate_stats(plddt)
    alphafold_results[key] = {
        'name': protein['name'],
        'plddt': plddt,
        'residues': res,
        'stats': stats
    }
    print(f"✅ Mean pLDDT: {stats['mean']:.1f}")

print("\n✅ AlphaFold analysis complete!")

## ESMFold Analysis

In [ ]:
def get_uniprot_sequence(uniprot_id: str) -> str:
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    lines = response.text.strip().split('\n')
    return ''.join(lines[1:])

def predict_esmfold(sequence: str, name: str) -> str:
    response = requests.post(ESMFOLD_API, data=sequence, timeout=300)
    response.raise_for_status()
    pdb_path = os.path.join(tempfile.gettempdir(), f"esm_{name.replace(' ', '_')}.pdb")
    with open(pdb_path, 'w') as f:
        f.write(response.text)
    return pdb_path

print("🟣 Running ESMFold Analysis...\n")
print("   ⏱️  Note: ESMFold prediction may take 1-2 minutes\n")
esmfold_results = {}

for key, protein in PROTEINS.items():
    print(f"   ESMFold: {protein['name']}...", end=' ')
    sequence = get_uniprot_sequence(protein['uniprot_id'])
    pdb_file = predict_esmfold(sequence, protein['name'])
    plddt, res = extract_plddt_from_pdb(pdb_file)
    stats = calculate_stats(plddt)
    esmfold_results[key] = {
        'name': protein['name'],
        'plddt': plddt,
        'residues': res,
        'stats': stats
    }
    print(f"✅ Mean pLDDT: {stats['mean']:.1f}")

print("\n✅ ESMFold analysis complete!")

## Comparison Summary

In [ ]:
print("="*70)
print("📊 MODEL COMPARISON SUMMARY")
print("="*70)

for key, protein in PROTEINS.items():
    af_stats = alphafold_results[key]['stats']
    esm_stats = esmfold_results[key]['stats']
    
    print(f"\n{protein['name']} ({protein['uniprot_id']}):")
    print(f"   🔵 AlphaFold:  {af_stats['mean']:.1f} ± {af_stats['std']:.1f} pLDDT")
    print(f"   🟣 ESMFold:    {esm_stats['mean']:.1f} ± {esm_stats['std']:.1f} pLDDT")
    print(f"   📈 Difference: {af_stats['mean'] - esm_stats['mean']:+.1f} points")

# Discrimination comparison
af_delta = alphafold_results['ubiquitin']['stats']['mean'] - alphafold_results['synuclein']['stats']['mean']
esm_delta = esmfold_results['ubiquitin']['stats']['mean'] - esmfold_results['synuclein']['stats']['mean']

print(f"\n{'='*70}")
print("Discrimination (Ubiquitin - Synuclein):")
print(f"   🔵 AlphaFold:  {af_delta:.1f} points")
print(f"   🟣 ESMFold:    {esm_delta:.1f} points")
print(f"   📊 Better at distinguishing: {'AlphaFold' if af_delta > esm_delta else 'ESMFold'}")

## 📊 Side-by-Side Comparison Visualization

In [ ]:
fig = plt.figure(figsize=(18, 12))

# Ubiquitin comparisons
ub_af = alphafold_results['ubiquitin']
ub_esm = esmfold_results['ubiquitin']

# Plot 1: Ubiquitin - AlphaFold
ax1 = plt.subplot(3, 3, 1)
ax1.plot(ub_af['residues'], ub_af['plddt'], linewidth=2, color='#2E86AB', label='AlphaFold')
ax1.axhline(y=90, color='green', linestyle='--', alpha=0.3)
ax1.axhline(y=70, color='orange', linestyle='--', alpha=0.3)
ax1.fill_between(ub_af['residues'], ub_af['plddt'], alpha=0.3, color='#2E86AB')
ax1.set_ylabel('pLDDT Score', fontsize=10)
ax1.set_title('Ubiquitin - AlphaFold', fontsize=11, fontweight='bold')
ax1.grid(alpha=0.3)
ax1.set_ylim([0, 105])

# Plot 2: Ubiquitin - ESMFold
ax2 = plt.subplot(3, 3, 2)
ax2.plot(ub_esm['residues'], ub_esm['plddt'], linewidth=2, color='#8B5CF6', label='ESMFold')
ax2.axhline(y=90, color='green', linestyle='--', alpha=0.3)
ax2.axhline(y=70, color='orange', linestyle='--', alpha=0.3)
ax2.fill_between(ub_esm['residues'], ub_esm['plddt'], alpha=0.3, color='#8B5CF6')
ax2.set_title('Ubiquitin - ESMFold', fontsize=11, fontweight='bold')
ax2.grid(alpha=0.3)
ax2.set_ylim([0, 105])

# Plot 3: Ubiquitin - Overlay
ax3 = plt.subplot(3, 3, 3)
ax3.plot(ub_af['residues'], ub_af['plddt'], linewidth=2, color='#2E86AB', label='AlphaFold', alpha=0.7)
ax3.plot(ub_esm['residues'], ub_esm['plddt'], linewidth=2, color='#8B5CF6', label='ESMFold', alpha=0.7)
ax3.axhline(y=90, color='green', linestyle='--', alpha=0.3)
ax3.axhline(y=70, color='orange', linestyle='--', alpha=0.3)
ax3.set_title('Ubiquitin - Overlay', fontsize=11, fontweight='bold')
ax3.legend(fontsize=9)
ax3.grid(alpha=0.3)
ax3.set_ylim([0, 105])

# Alpha-synuclein comparisons
syn_af = alphafold_results['synuclein']
syn_esm = esmfold_results['synuclein']

# Plot 4: Synuclein - AlphaFold
ax4 = plt.subplot(3, 3, 4)
ax4.plot(syn_af['residues'], syn_af['plddt'], linewidth=2, color='#2E86AB')
ax4.axhline(y=90, color='green', linestyle='--', alpha=0.3)
ax4.axhline(y=70, color='orange', linestyle='--', alpha=0.3)
ax4.fill_between(syn_af['residues'], syn_af['plddt'], alpha=0.3, color='#2E86AB')
ax4.set_ylabel('pLDDT Score', fontsize=10)
ax4.set_title('Alpha-synuclein - AlphaFold', fontsize=11, fontweight='bold')
ax4.grid(alpha=0.3)
ax4.set_ylim([0, 105])

# Plot 5: Synuclein - ESMFold
ax5 = plt.subplot(3, 3, 5)
ax5.plot(syn_esm['residues'], syn_esm['plddt'], linewidth=2, color='#8B5CF6')
ax5.axhline(y=90, color='green', linestyle='--', alpha=0.3)
ax5.axhline(y=70, color='orange', linestyle='--', alpha=0.3)
ax5.fill_between(syn_esm['residues'], syn_esm['plddt'], alpha=0.3, color='#8B5CF6')
ax5.set_xlabel('Residue Number', fontsize=10)
ax5.set_title('Alpha-synuclein - ESMFold', fontsize=11, fontweight='bold')
ax5.grid(alpha=0.3)
ax5.set_ylim([0, 105])

# Plot 6: Synuclein - Overlay
ax6 = plt.subplot(3, 3, 6)
ax6.plot(syn_af['residues'], syn_af['plddt'], linewidth=2, color='#2E86AB', label='AlphaFold', alpha=0.7)
ax6.plot(syn_esm['residues'], syn_esm['plddt'], linewidth=2, color='#8B5CF6', label='ESMFold', alpha=0.7)
ax6.axhline(y=90, color='green', linestyle='--', alpha=0.3)
ax6.axhline(y=70, color='orange', linestyle='--', alpha=0.3)
ax6.set_xlabel('Residue Number', fontsize=10)
ax6.set_title('Alpha-synuclein - Overlay', fontsize=11, fontweight='bold')
ax6.legend(fontsize=9)
ax6.grid(alpha=0.3)
ax6.set_ylim([0, 105])

# Plot 7: Model comparison bar chart - Ubiquitin
ax7 = plt.subplot(3, 3, 7)
models = ['AlphaFold', 'ESMFold']
means = [ub_af['stats']['mean'], ub_esm['stats']['mean']]
ax7.bar(models, means, color=['#2E86AB', '#8B5CF6'], alpha=0.7)
ax7.set_ylabel('Mean pLDDT', fontsize=10)
ax7.set_title('Ubiquitin - Model Comparison', fontsize=11, fontweight='bold')
ax7.grid(alpha=0.3, axis='y')
for i, v in enumerate(means):
    ax7.text(i, v + 1, f'{v:.1f}', ha='center', fontsize=9)

# Plot 8: Model comparison bar chart - Synuclein
ax8 = plt.subplot(3, 3, 8)
means = [syn_af['stats']['mean'], syn_esm['stats']['mean']]
ax8.bar(models, means, color=['#2E86AB', '#8B5CF6'], alpha=0.7)
ax8.set_title('Synuclein - Model Comparison', fontsize=11, fontweight='bold')
ax8.grid(alpha=0.3, axis='y')
for i, v in enumerate(means):
    ax8.text(i, v + 1, f'{v:.1f}', ha='center', fontsize=9)

# Plot 9: Discrimination comparison
ax9 = plt.subplot(3, 3, 9)
discriminations = [af_delta, esm_delta]
ax9.bar(models, discriminations, color=['#2E86AB', '#8B5CF6'], alpha=0.7)
ax9.set_ylabel('Δ pLDDT (Ubiq - Synuc)', fontsize=10)
ax9.set_title('Discrimination Power', fontsize=11, fontweight='bold')
ax9.grid(alpha=0.3, axis='y')
for i, v in enumerate(discriminations):
    ax9.text(i, v + 0.5, f'{v:.1f}', ha='center', fontsize=9)

plt.suptitle('AlphaFold vs ESMFold: Comprehensive Comparison', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n✅ Comparison visualization complete!")

## Save Comparison Results

In [ ]:
comparison_results = {
    'proteins': PROTEINS,
    'alphafold': {key: {'name': val['name'], 'stats': val['stats']} for key, val in alphafold_results.items()},
    'esmfold': {key: {'name': val['name'], 'stats': val['stats']} for key, val in esmfold_results.items()},
    'comparison': {
        'alphafold_discrimination': af_delta,
        'esmfold_discrimination': esm_delta,
        'better_model': 'AlphaFold' if af_delta > esm_delta else 'ESMFold'
    }
}

with open('alphafold_vs_esmfold_comparison.json', 'w') as f:
    json.dump(comparison_results, f, indent=2)

print("💾 Saved: alphafold_vs_esmfold_comparison.json")
print("\n✅ COMPLETE COMPARISON FINISHED!")
print("\n📊 Key Insights:")
print(f"   • AlphaFold discrimination: {af_delta:.1f} points")
print(f"   • ESMFold discrimination: {esm_delta:.1f} points")
print(f"   • Better at distinguishing: {comparison_results['comparison']['better_model']}")
print("\n🎯 Both models successfully identify structured vs disordered proteins!")